In [1]:
from nbed.driver import NbedDriver
from pathlib import Path

args = {
    "geometry": str(Path("molecular_structures/cyclopentane.xyz")),
    "basis": "STO-3G",
    "xc_functional": "b3lyp",
    "n_active_atoms":2,
    "projector": "mu",
    "localization": "spade",
    "convergence": 1e-6,
    "savefile": None,
    "run_ccsd_emb": True,
    "run_fci_emb": True,
    'charge':0,
    'spin':0,
}

driver = NbedDriver(
    **args,
)
#     geometry=args["geometry"],
#     n_active_atoms=args['n_active_atoms'],
#     basis=args["basis"],
#     xc_functional=args["xc_functional"],
#     projector=args["projector"],
#     localization=args["localization"],
#     convergence=args["convergence"],
#     savefile=args["savefile"],
#     run_ccsd_emb=args["run_ccsd_emb"],
#     run_fci_emb=args["run_fci_emb"],
# )

/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'
/home/mwilliams/anaconda3/envs/nbed/lib/python3.8/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
import logging
logger = logging.getLogger(__name__)
from nbed.localizers import Localizer
from typing import Optional, Tuple, Dict
import numpy as np
from pyscf import scf, dft, gto

init_huzinaga_rhf_with_mu = False

"""Generate embedded Hamiltonian.

Note run_mu_shift (bool) and run_huzinaga (bool) flags define which method to use (can be both)
This is done when object is initialized.
"""
logger.debug("Embedding molecule.")
localized_system = driver._localize()
driver.localized_system = localized_system

# logger.info(localized_system.active_MO_inds, localized_system.beta_active_MO_inds)
# logger.info(localized_system.enviro_MO_inds, localized_system.beta_enviro_MO_inds)

e_nuc = driver._global_ks.energy_nuc()

# Run subsystem DFT (calls localized rks)
#driver._subsystem_dft()


In [5]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _ks_components(
    ks_system: Localizer,
    subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.

    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.

    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)

    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    logger.debug("Finding subsystem RKS componenets.")
    # It seems that PySCF lumps J and K in the J array
    # need to access the potential for the right subsystem for unrestricted
    two_e_term = ks_system.get_veff(dm=subsystem_dm)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", ks_system.get_hcore(), subsystem_dm)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")
    logger.debug(f"Subsystem RKS components found.")
    return energy_elec, e_xc, j_mat

(alpha_e_act, alpha_e_xc_act, alpha_j_act) = _ks_components(
    driver._global_ks, driver.localized_system.dm_active
)
print(alpha_e_act, alpha_e_xc_act)
(alpha_e_env, alpha_e_xc_env, alpha_j_env) = _ks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
print(alpha_e_env, alpha_e_xc_env)
driver.e_act = alpha_e_act
driver.e_env = alpha_e_env

if not driver._restricted_scf:
    (beta_e_act, beta_e_xc_act, beta_j_act) = _ks_components(
        driver._global_ks, driver.localized_system.beta_dm_active
    )
    print(beta_e_act, beta_e_xc_act)
    (beta_e_env, beta_e_xc_env, beta_j_env) = _ks_components(
        driver._global_ks, driver.localized_system.beta_dm_enviro
    )
    print(beta_e_env, beta_e_xc_env)
    driver.e_act = (driver.e_act + beta_e_act)/2
    driver.e_env = (driver.e_env + beta_e_env)/2

# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")
total_dm = driver.localized_system.dm_active + driver.localized_system.dm_enviro
if not driver._restricted_scf:
    total_dm += driver.localized_system.beta_dm_active + driver.localized_system.beta_dm_enviro
    
two_e_term_total = driver._global_ks.get_veff(
    dm=total_dm)
e_xc_total = two_e_term_total.exc

# if not driver._restricted_scf:
#     beta_two_e_term_total = driver._global_ks.get_veff(
#         dm=driver.localized_system.beta_dm_active + driver.localized_system.beta_dm_enviro
#     )
#     e_xc_total = (e_xc_total + beta_two_e_term_total.exc)/2

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, alpha_j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, alpha_j_act)
)
if not driver._restricted_scf:
    beta_j_cross = 0.5 * (
        np.einsum("ij,ij", driver.localized_system.dm_active, beta_j_env)
        + np.einsum("ij,ij", driver.localized_system.dm_enviro, beta_j_act)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_active, beta_j_env)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_enviro, beta_j_act)
        + np.einsum("ij,ij", driver.localized_system.beta_dm_active, alpha_j_env)  
        + np.einsum("ij,ij", driver.localized_system.beta_dm_enviro, alpha_j_act)
    )
    j_cross += beta_j_cross

# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - alpha_e_xc_act - alpha_e_xc_env
# if not driver._restricted_scf:
    # xc_cross = beta_e_xc_act - beta_e_xc_env
print(e_xc_total, alpha_e_xc_act, alpha_e_xc_env)
if not driver._restricted_scf:
    print(beta_e_xc_act, beta_e_xc_env)

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross
print(f"{j_cross} {k_cross} {xc_cross}")

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
logger.info("RKS components")
logger.info(driver.e_act)
logger.info(driver.e_env)
logger.info(driver.two_e_cross)
logger.info(driver._global_ks.energy_nuc())
print(energy_DFT_components)
print(driver._global_ks.e_tot)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):
    logger.error(
        "DFT energy of localized components not matching supersystem DFT."
    )
    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT."
    )


-53.31904648524806 -2.6015469424991795
0.0 0.0


1970-01-01 09:17:22,279: __main__: ERROR: DFT energy of localized components not matching supersystem DFT.


-2.6015469424991795 -2.6015469424991795 0.0
0.0 0.0 0.0
135.78876774003754
-194.0797485446971


ValueError: DFT energy of localized components not matching supersystem DFT.

In [6]:
"""Function to perform subsystem RKS DFT calculation."""
logger.debug("Calculating active and environment subsystem terms.")

def _rks_components(
    rks_system: Localizer, subsystem_dm: np.ndarray,
) -> Tuple[float, float, np.ndarray, np.ndarray, np.ndarray]:
    """Calculate the components of subsystem energy from a RKS DFT calculation.
    For a given density matrix this function returns the electronic energy, exchange correlation energy and
    J,K, V_xc matrices.
    Args:
        dm_matrix (np.ndarray): density matrix (to calculate all matrices from)
    Returns:
        Energy_elec (float): DFT energy defubed by input density matrix
        e_xc (float): exchange correlation energy defined by input density matrix
        J_mat (np.ndarray): J_matrix defined by input density matrix
    """
    dm_matrix = subsystem_dm
    # It seems that PySCF lumps J and K in the J array
    two_e_term = rks_system.get_veff(dm=dm_matrix)
    j_mat = two_e_term.vj
    # k_mat = np.zeros_like(j_mat)

    e_xc = two_e_term.exc
    # v_xc = two_e_term - j_mat

    energy_elec = (
        np.einsum("ij,ji->", rks_system.get_hcore(), dm_matrix)
        + two_e_term.ecoul
        + two_e_term.exc
    )

    # if check_E_with_pyscf:
    #     energy_elec_pyscf = driver._global_ks.energy_elec(dm=dm_matrix)[0]
    #     if not np.isclose(energy_elec_pyscf, energy_elec):
    #         raise ValueError("Energy calculation incorrect")

    return energy_elec, e_xc, j_mat

(driver.e_act, e_xc_act, j_act) = _rks_components(
    driver._global_ks, driver.localized_system.dm_active
)
(driver.e_env, e_xc_env, j_env) = _rks_components(
    driver._global_ks, driver.localized_system.dm_enviro
)
# Computing cross subsystem terms
logger.debug("Calculating two electron cross subsystem energy.")

two_e_term_total = driver._global_ks.get_veff(
    dm=driver.localized_system.dm_active + driver.localized_system.dm_enviro
)
e_xc_total = two_e_term_total.exc

j_cross = 0.5 * (
    np.einsum("ij,ij", driver.localized_system.dm_active, j_env)
    + np.einsum("ij,ij", driver.localized_system.dm_enviro, j_act)
)
# Because of projection
k_cross = 0.0

xc_cross = e_xc_total - e_xc_act - e_xc_env

# overall two_electron cross energy
driver.two_e_cross = j_cross + k_cross + xc_cross

energy_DFT_components = (
    driver.e_act + driver.e_env + driver.two_e_cross + driver._global_ks.energy_nuc()
)
logger.info("RKS components")
logger.info(driver.e_act)
logger.info(driver.e_env)
logger.info(driver.two_e_cross)
logger.info(driver._global_ks.energy_nuc())
print(energy_DFT_components, driver._global_ks.e_tot)
if not np.isclose(energy_DFT_components, driver._global_ks.e_tot):

    raise ValueError(
        "DFT energy of localized components not matching supersystem DFT"
    )

135.78876774003754 -194.0797485446971


ValueError: DFT energy of localized components not matching supersystem DFT

In [ ]:

logger.debug("Getting global DFT potential to optimize embedded calc in.")
g_act_and_env = driver._global_ks.get_veff(
    dm=(localized_system.dm_active + localized_system.dm_enviro)
)
g_act = driver._global_ks.get_veff(dm=localized_system.dm_active)
driver._dft_potential = g_act_and_env - g_act
logger.info(f"DFT potential average {np.mean(driver._dft_potential)}.")

# To add a projector, put it in this dict with a function
# if we want any more it's also time to turn them into a class
embeddings: Dict[str, callable] = {
    "huzinaga": driver._huzinaga_embed,
    "mu": driver._mu_embed,
}
if driver.projector in embeddings:
    embeddings = {driver.projector: embeddings[driver.projector]}

# This is reverse so that huz can be initialised with mu
for name in sorted(embeddings, reverse=True):
    logger.debug(f"Runnning embedding with {name} projector.")
    setattr(driver, "_" + name, {})
    result = getattr(driver, "_" + name)

    embedding_method = embeddings[name]
    local_rhf = driver._init_local_rhf()

    if init_huzinaga_rhf_with_mu and (name == "huzinaga"):
        logger.debug("Initializing huzinaga with mu-shift.")
        # seed huzinaga calc with mu result!
        result["v_emb"], result["scf"] = embedding_method(
            local_rhf, dmat_initial_guess=driver._mu["scf"].make_rdm1()
        )
    else:
        result["v_emb"], result["scf"] = embedding_method(local_rhf)

    result["mo_energies_emb_pre_del"] = local_rhf.mo_energy
    result["scf"] = driver._delete_environment(result["scf"], name)
    result["mo_energies_emb_post_del"] = local_rhf.mo_energy

    logger.info(f"V emb mean {name}: {np.mean(result['v_emb'])}")

    # calculate correction
    result["correction"] = np.einsum(
        "ij,ij", result["v_emb"], localized_system.dm_active
    )
    result["e_rhf"] = (
        result["scf"].e_tot
        + driver.e_env
        + driver.two_e_cross
        - result["correction"]
    )
    logger.info(f"RHF energy: {result['e_rhf']}")

    # classical energy
    result["classical_energy"] = (
        driver.e_env + driver.two_e_cross + e_nuc - result["correction"]
    )

    # Calculate ccsd or fci energy
    if driver.run_ccsd_emb is True:
        logger.debug("Performing CCSD-in-DFT embedding.")
        ccsd_emb, e_ccsd_corr = driver._run_emb_CCSD(
            result["scf"], frozen_orb_list=None
        )
        result["e_ccsd"] = (
            ccsd_emb.e_hf
            + e_ccsd_corr
            + driver.e_env
            + driver.two_e_cross
            - result["correction"]
        )
        logger.info(f"CCSD Energy {name}:\t{result['e_ccsd']}")

    if driver.run_fci_emb is True:
        logger.debug("Performing FCI-in-DFT embedding.")
        fci_emb = driver._run_emb_FCI(result["scf"], frozen_orb_list=None)
        result["e_fci"] = (
            (fci_emb.e_tot)
            + driver.e_env
            + driver.two_e_cross
            - result["correction"]
        )
        logger.info(f"FCI Energy {name}:\t{result['e_fci']}")

    if driver.run_dft_in_dft is True:
        did = driver.embed_dft_in_dft(driver._global_ks.xc, embedding_method)
        result["e_dft_in_dft"] = did["e_rks"]

if driver.projector == "both":
    logger.warning(
        "Outputting both mu and huzinaga embedding results as tuple."
    )
    driver.embedded_scf = (
        driver._mu["scf"],
        driver._huzinaga["scf"],
    )
    driver.classical_energy = (
        driver._mu["classical_energy"],
        driver._huzinaga["classical_energy"],
    )
elif driver.projector == "mu":
    driver.embedded_scf = driver._mu["scf"]
    driver.classical_energy = driver._mu["classical_energy"]
elif driver.projector == "huzinaga":
    driver.embedded_scf = driver._huzinaga["scf"]
    driver.classical_energy = driver._huzinaga["classical_energy"]

logger.info("Embedding complete.")